RED CAR OPTICAL FLOW TRACK 

In [ ]:
import numpy as np
import cv2 as cv
import os

# Remplacez ici par le chemin de votre fichier vidéo
video_path = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\TP2 optical flow\redcar.mp4"

# Vérifiez que le chemin de sortie existe (ou créez-le)
output_dir = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\output"
os.makedirs(output_dir, exist_ok=True)

# Ouvrir la vidéo
cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Erreur : Impossible d'ouvrir la vidéo {video_path}")
    exit()

# Paramètres pour la détection des coins Shi-Tomasi
feature_params = dict(maxCorners=50, qualityLevel=0.3, minDistance=5, blockSize=5)

# Paramètres pour le calcul du flux optique Lucas Kanade
lk_params = dict(winSize=(21, 21), maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.003))

# Couleurs aléatoires pour tracer les flèches (10 points)
color = np.random.randint(0, 255, (10, 3))

# Lire la première image de la vidéo et trouver les coins
ret, old_frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la première image de la vidéo.")
    cap.release()
    exit()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Masque pour dessiner les trajectoires
mask = np.zeros_like(old_frame)

# Compteur pour l'intervalle
frame_count = 0
save_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo.")
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculer le flux optique
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Tracer les flèches pour représenter le flux optique
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()  # Nouvelle position
            c, d = old.ravel()  # Ancienne position
            # Dessiner une flèche proportionnelle au déplacement
            mask = cv.arrowedLine(mask, (int(c), int(d)), (int(a), int(b)),
                                  color[i % len(color)].tolist(), thickness=1, tipLength=0.4)

            # Dessiner un cercle pour indiquer le point actuel
            frame = cv.circle(frame, (int(a), int(b)), 3, color[i % len(color)].tolist(), -1)

        img = cv.add(frame, mask)

        # Afficher le résultat
        cv.imshow('Flux Optique - Flèches Déplacement', img)

        # Appuyer sur 'C' pour sauvegarder l'image et extraire les paramètres
        key = cv.waitKey(30) & 0xFF
        if key == ord('c'):
            save_count += 1
            save_path = os.path.join(output_dir, f"saved_frame_{save_count}.jpg")
            cv.imwrite(save_path, img)
            print(f"Image sauvegardée : {save_path}")

            # Extraire les paramètres
            displacements = good_new - good_old
            magnitudes = np.linalg.norm(displacements, axis=1)
            angles = np.arctan2(displacements[:, 1], displacements[:, 0]) * (180 / np.pi)

            # Calculer les statistiques
            magnitude_mean = np.mean(magnitudes)
            angle_mean = np.mean(angles)
            density = len(good_new)
            distance_mean = np.mean(magnitudes)

            # Afficher les résultats
            print("\n=== Paramètres du Flux Optique ===")
            print(f"Image sauvegardée : {save_path}")
            print(f"Magnitude Moyenne : {magnitude_mean:.2f}")
            print(f"Direction Moyenne : {angle_mean:.2f}°")
            print(f"Densité des Points : {density}")
            print(f"Distance Moyenne Parcourue : {distance_mean:.2f}")
            print(f"Magnitudes individuelles : {magnitudes}")
            print(f"Directions individuelles (en degrés) : {angles}")
            print("=================================\n")

        # Quitter avec 'Esc'
        if key == 27:
            break

        # Mise à jour des frames et points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
        frame_count += 1
    else:
        print("Aucun point valide détecté.")
        break

# Libérer les ressources
cap.release()
cv.destroyAllWindows()

BEE Track 10 points

In [2]:
import numpy as np
import cv2 as cv
import os

# Remplacez ici par le chemin de votre fichier vidéo
video_path = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\TP2 optical flow\beysSmaller.mp4"

# Vérifiez que le chemin de sortie existe (ou créez-le)
output_dir = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\output"
os.makedirs(output_dir, exist_ok=True)

# Ouvrir la vidéo
cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Erreur : Impossible d'ouvrir la vidéo {video_path}")
    exit()

# Paramètres pour la détection des coins Shi-Tomasi
feature_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)

# Paramètres pour le calcul du flux optique Lucas-Kanade
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Couleurs aléatoires pour tracer les trajectoires
color = np.random.randint(0, 255, (100, 3))

# Lire la première image de la vidéo et trouver les coins
ret, old_frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la première image de la vidéo.")
    cap.release()
    exit()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Masque pour dessiner les trajectoires
mask = np.zeros_like(old_frame)

# Compteur pour sauvegarder les images
save_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo.")
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculer le flux optique
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Tracer les trajectoires
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv.add(frame, mask)

        # Afficher le résultat
        cv.imshow('Flux Optique - Lucas Kanade', img)

        # Appuyer sur 'C' pour sauvegarder l'image avec les points
        key = cv.waitKey(30) & 0xFF
        if key == ord('c'):
            save_count += 1
            save_path = os.path.join(output_dir, f"saved_frame_{save_count}.jpg")
            cv.imwrite(save_path, img)
            print(f"Image sauvegardée : {save_path}")

        # Quitter avec 'Esc'
        if key == 27:
            break

        # Mettre à jour pour la prochaine itération
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    else:
        print("Aucun point valide détecté.")
        break

# Libérer les ressources
cap.release()
cv.destroyAllWindows()


BEE Track 100 points

In [1]:
import numpy as np
import cv2 as cv
import os

# Remplacez ici par le chemin de votre fichier vidéo
video_path = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\TP2 optical flow\beysSmaller.mp4"

# Vérifiez que le chemin de sortie existe (ou créez-le)
output_dir = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\output"
os.makedirs(output_dir, exist_ok=True)

# Ouvrir la vidéo
cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Erreur : Impossible d'ouvrir la vidéo {video_path}")
    exit()

# Paramètres pour la détection des coins Shi-Tomasi
feature_params = dict(maxCorners=100, qualityLevel=0.1, minDistance=7, blockSize=7)

# Paramètres pour le calcul du flux optique Lucas-Kanade
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Couleurs aléatoires pour tracer les trajectoires
color = np.random.randint(0, 255, (100, 3))

# Lire la première image de la vidéo et trouver les coins
ret, old_frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la première image de la vidéo.")
    cap.release()
    exit()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Masque pour dessiner les trajectoires
mask = np.zeros_like(old_frame)

# Compteur pour sauvegarder les images
save_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin de la vidéo.")
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculer le flux optique
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Tracer les trajectoires
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv.add(frame, mask)

        # Afficher le résultat
        cv.imshow('Flux Optique - Lucas Kanade', img)

        # Appuyer sur 'C' pour sauvegarder l'image avec les points
        key = cv.waitKey(30) & 0xFF
        if key == ord('c'):
            save_count += 1
            save_path = os.path.join(output_dir, f"saved_frame_{save_count}.jpg")
            cv.imwrite(save_path, img)
            print(f"Image sauvegardée : {save_path}")

        # Quitter avec 'Esc'
        if key == 27:
            break

        # Mettre à jour pour la prochaine itération
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    else:
        print("Aucun point valide détecté.")
        break

# Libérer les ressources
cap.release()
cv.destroyAllWindows()


RED CAR DENSE OPTICAL FLOW

In [ ]:
import numpy as np
import cv2 as cv

video_path = r"C:\Users\utilisateur\Desktop\Cours\SEQUENCE VIDEO\TP2 optical flow\redcar.mp4"

# Ouvrir la vidéo
cap = cv.VideoCapture(video_path)

# Vérifiez si la webcam s'ouvre correctement
if not cap.isOpened():
    print("Erreur : Impossible d'accéder à la webcam.")
    exit()

# Lecture de la première image de la webcam
ret, frame1 = cap.read()
if not ret:
    print("Erreur : Impossible de lire une image de la webcam.")
    exit()

prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

# Compteur pour nommer les fichiers sauvegardés
save_count = 0

while True:
    ret, frame2 = cap.read()
    if not ret:
        print("Erreur : Impossible de lire une image de la webcam.")
        break

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    cv.imshow('Flux optique (Optical Flow)', bgr)
    cv.imshow('Image normale', frame2)

    # Gestion des entrées clavier
    k = cv.waitKey(30) & 0xFF
    if k == 27:  # Touche 'Échap' pour quitter
        break
    elif k == ord('c'):  # Sauvegarder les images si la touche 'c' est pressée
        save_count += 1
        cv.imwrite(f'optical_flow_{save_count}.png', bgr)  # Sauvegarder le flux optique
        cv.imwrite(f'normal_image_{save_count}.png', frame2)  # Sauvegarder l'image normale
        print(f"Images sauvegardées : optical_flow_{save_count}.png et normal_image_{save_count}.png")

    prvs = next

cap.release()
cv.destroyAllWindows()
